In [1]:
from sim.model import simple_sim
from sim.sum_stats import simple_sum
import logging
import time
import pandas as pd
import numpy as np
import os
import pickle
import priors # This runs priors.py so we can read in the priors

logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)

<Figure size 432x288 with 0 Axes>

In [2]:
start_time = time.time()
# array_id = int(os.environ['PBS_ARRAYID'])
array_id = 1
print("WARNING NEED TO REPLACE ARRAY ID")

WARNING NEED TO REPLACE ARRAY ID


In [3]:
runs_per_task = 3    # Jobs in an array are referred to as tasks
start_index = array_id*runs_per_task
end_index = start_index+runs_per_task

In [4]:
prior_df = pd.read_feather("../output/rejection/priors.feather")

In [5]:
failed_runs = 0

# Run the model runs_per_task times
for i in range(start_index, end_index):
    np.random.seed(i)
    params = prior_df.astype(object).iloc[i].to_dict()  # Row of parameters
    

    try:  # Just to make sure results end up being NA instead of breaking the run.
        data = simple_sim(**params, length = int(5e6),
                      recombination_rate = 1.8e-8,
                      mutation_rate = 6e-8, seed=i)  # Run model with params

    except Exception:
        logging.warning("The simulation failed to run on parameter index {}".format(i))
        failed_runs += 1
        if failed_runs == 2:
            raise ValueError('The number of failed runs in this batch reached 3!')
        continue  # If something goes wrong go to next iteration (row will be np.nan)

    stats_dict = simple_sum(data)

    logging.info("Calculating summary stats finished in {:.2f} s".format(time.time() - start_time))

    if i == start_index:  # Saves writing out all the parameter names to initiate the df beforehand
        stats_df = pd.DataFrame(np.nan, index=range(start_index, end_index), columns=stats_dict.keys())
        stats_df.loc[i] = list(stats_dict.values())
    else:
        for col, value in stats_dict.items():
            stats_df.loc[i, col] = value

stats_df = stats_df.reset_index(drop=True)
output_filepath = "../output/rejection/summary_stats/summary_stats_{}.feather".format(array_id)
stats_df.to_feather(output_filepath)


INFO:root:Param bottleneck_strength_domestic = 8162
INFO:root:Param bottleneck_strength_wild = 9239
INFO:root:Param bottleneck_time_domestic = 3754
INFO:root:Param bottleneck_time_wild = 3756
INFO:root:Param captive_time = 15
INFO:root:Param div_time = 36737
INFO:root:Param mig_length_post_split = 2063
INFO:root:Param mig_length_wild = 10
INFO:root:Param mig_rate_captive = 0.1840646910664301
INFO:root:Param mig_rate_post_split = 0.19540467068879347
INFO:root:Param mig_rate_wild = 0.09669319125962743
INFO:root:Param pop_size_captive = 152
INFO:root:Param pop_size_domestic_1 = 8792
INFO:root:Param pop_size_domestic_2 = 10340
INFO:root:Param pop_size_wild_1 = 5950
INFO:root:Param pop_size_wild_2 = 5669
INFO:root:Running command: slim -d captive_time=15 -d mig_length_wild=10 -d mig_rate_captive=0.1840646910664301 -d mig_rate_wild=0.09669319125962743 -d pop_size_captive=152 -d pop_size_domestic_1=8792 -d pop_size_wild_1=5950 -d length=5000000 -d recombination_rate=1.8e-08  -d decap_trees_fi